# `CLOUD.jl` - 2D linear advection example - energy-conservative scheme in collapsed coordinates

In [7]:
using CLOUD, OrdinaryDiffEq

Define the physical problem

In [8]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [9]:
M = 2
p = 8
p_map = 8
form = WeakConservationForm(mapping_form=SkewSymmetricMapping(),
    inviscid_numerical_flux=LaxFriedrichsNumericalFlux(0.0))
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.01*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `CollapsedSEM` scheme of degree `p` with `M` edges in each direction

In [10]:
reference_approximation =ReferenceApproximation(
    CollapsedSEM(p), Tri(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [11]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:          713μs / 1374183.0%      10.6KiB / 87277041.0%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual           192k    8.68s   88.6%  45.1μs   6.65GiB   75.6%  36.3KiB
   volume terms          192k    6.44s   65.7%  33.5μs   5.51GiB   62.7%  30.0KiB
   mass matrix solve     192k    1.60s   16.3%  8.31μs    270MiB    3.0%  1.44KiB
   facet terms           192k    364ms    3.7%  1.89μs    578MiB    6.4%  3.08KiB
 extrap solution         192k    541ms    5.5%  2.81μs    543MiB    6.0%  2.89KiB
 eval num flux           192k    247ms    2.5%  1.28μs    605MiB    6.7%  3.22KiB
 eval flux     

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [12]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0030761191433130676
Conservation (initial/final/diff):
[1.3620120437735084e-8][1.3620120045687578e-8][-3.920475055707584e-16]
Energy (initial/final/diff):
[0.12499958214791701][0.12499958214791601][-9.992007221626409e-16]
